In [1]:
import pandas as pd
import numpy as np
import os
import FinanceDataReader as fdr
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from pandas import datetime
from pmdarima.arima import auto_arima
from sklearn.model_selection import GridSearchCV
import statsmodels.api as sm
import warnings

<ipython-input-1-bec2a2f20ba2>:10: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [2]:
# read stock code, list

path = './open'
list_name = 'Stock_List.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
sample_submission = pd.read_csv(os.path.join(path,sample_name))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [3]:
# start_date = '20210101'
start_date = '20210101'
end_date = '20211031'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())
display(Business_days.tail())

sample_submission = pd.read_csv(os.path.join(path,sample_name))

WEEKDAY of "start_date" : 4
NUM of WEEKS to "end_date" : 43
HOW MANY "Business_days" : (216, 1)


,Date
0,2021-01-01
1,2021-01-04
2,2021-01-05
3,2021-01-06
4,2021-01-07


,Date
211,2021-10-25
212,2021-10-26
213,2021-10-27
214,2021-10-28
215,2021-10-29


In [4]:
def get_predict(x_close):
    
    param_grid = {
        'order': [(0,1,1),(1,1,0),(1,1,1),(2,1,0),(3,1,0),(4,1,0),(0,1,2),(0,1,3),(0,1,4)],
        'criterion': ["mse", "mae"],
    }

    grid_search = GridSearchCV(ARIMA(x_close, order=(0,1,0)), param_grid=param_grid, n_jobs=-1)
    grid_search.fit(x_close)
    best_grid = grid_search.best_estimator_
    prediction_close = best_grid.predcit(steps=5)

    return prediction_close

In [5]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
	# prepare training dataset
	train_size = len(X)-5
	train, test = X[0:train_size], X[train_size:]

	# make predictions
	# predictions = []

	model = ARIMA(train, order=arima_order)
	model_fit = model.fit()
	# yhat = model_fit.forecast(steps=5)
	# yhat=list(yhat)
	# test=list(test)
	# predictions.append(yhat)
	# # calculate out of sample error
	# error = mean_squared_error(test, yhat)
	return model_fit.aic

In [6]:

# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
	dataset = dataset.astype('float32')
	best_score, best_cfg = float("inf"), None
	for p in p_values:
		for d in d_values:
			for q in q_values:
				order = (p,d,q)

				try:
					mse = evaluate_arima_model(dataset, order)
					if mse > 1000:
						if mse < best_score:
							best_score, best_cfg = mse, order
							print('ARIMA%s AIC=%.3f' % (order,mse))
				except:
					continue
	print('Best ARIMA%s AIC=%.3f' % (best_cfg, best_score))
	

	model = ARIMA(dataset, order=best_cfg)
	model_fit = model.fit()
	prediction = model_fit.forecast(steps=5)
	print(list(prediction))
	return prediction

In [7]:
p_values=[0,1,2,3]
d_values=[1,2]
q_values=[0,1,2,3]
warnings.filterwarnings("ignore")
sample_code = stock_list.loc[0,'종목코드']

sample = fdr.DataReader(sample_code, start = start_date, end = end_date)[['Close']].reset_index()
sample = pd.merge(Business_days, sample, how = 'outer')
sample.Close = sample.Close.ffill()
sample.Close = sample.Close.bfill()
sample_data = sample['Close'][:]
# sample_data = list(sample_data)
print(sample_data)


prediction=evaluate_models(sample_data,p_values,d_values,q_values)    
prediction=list(prediction)
prediction



0      83000.0
1      83000.0
2      83900.0
3      82200.0
4      82900.0
        ...   
211    70200.0
212    71100.0
213    70100.0
214    70700.0
215    69800.0
Name: Close, Length: 216, dtype: float64
ARIMA(0, 1, 0) AIC=3529.414
ARIMA(2, 1, 2) AIC=3509.505
Best ARIMA(2, 1, 2) AIC=3509.505
[69817.87679461033, 69950.98235576077, 69904.74943178183, 69789.96703501664, 69858.7781448329]


[69817.87679461033,
 69950.98235576077,
 69904.74943178183,
 69789.96703501664,
 69858.7781448329]

In [8]:
# model = LinearRegression()
p_values=[0,1,2,3]
d_values=[1,2]
q_values=[0,1,2,3]
warnings.filterwarnings("ignore")

for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data.Close = data.Close.ffill()
    data.Close = data.Close.bfill()
    data.index = data["Date"]
    arima_data = data['Close'][:]
    # arima_data = list(arima_data)
    

    predictions=evaluate_models(arima_data,p_values,d_values,q_values)    
    predictions=list(predictions)
    # print(predictions)
    # forecast = get_predict(arima_data) #(0,1,0)인수 나오는것 처리해야함
    # predictions = list(forecast)
    # print(predictions)

    sample_submission.loc[0:4,code] = predictions 
sample_submission.isna().sum().sum()

  0%|          | 0/370 [00:00<?, ?it/s]

ARIMA(0, 1, 0) AIC=3529.414
ARIMA(2, 1, 2) AIC=3509.505
Best ARIMA(2, 1, 2) AIC=3509.505


  0%|          | 1/370 [00:08<55:01,  8.95s/it]

[69817.87679461033, 69950.98235576077, 69904.74943178183, 69789.96703501664, 69858.7781448329]
ARIMA(0, 1, 0) AIC=3928.297


  1%|          | 2/370 [00:17<53:18,  8.69s/it]

...

100%|█████████▉| 369/370 [47:25<00:07,  7.33s/it]

Best ARIMA(0, 1, 0) AIC=3473.265
[42850.0, 42850.0, 42850.0, 42850.0, 42850.0]
ARIMA(0, 1, 0) AIC=3835.544
ARIMA(0, 1, 3) AIC=3835.058
ARIMA(2, 1, 2) AIC=3830.276
ARIMA(2, 1, 3) AIC=3830.081
Best ARIMA(2, 1, 3) AIC=3830.081


100%|██████████| 370/370 [47:31<00:00,  7.71s/it]

[50122.589176369875, 50340.55982641156, 50439.564730706275, 50269.12839944403, 50197.091256743995]


0

In [9]:
sample_submission

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,...,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,27537.914115,34800.0,59974.878579,141714.407640,103000.0,16662.046560,47133.794342,85670.437427,103000.0,...,49262.631196,52145.424434,82300.0,35881.968872,25719.971918,53337.729710,335000.0,261561.328844,27153.174713,17090.161904
1,2021-11-02,27742.684358,34800.0,60108.296456,141922.323942,103000.0,16680.166558,46976.254603,85592.444685,103000.0,...,49026.341177,52261.574028,82300.0,35713.902408,25797.340853,53529.564979,335000.0,262163.426061,27174.982867,17089.194023
2,2021-11-03,27802.818237,34800.0,60063.370424,142333.739391,103000.0,16734.130852,46916.180771,85112.195029,103000.0,...,48828.941605,52218.410411,82300.0,35561.309722,25874.360554,53597.651799,335000.0,262425.821417,27215.049381,17089.098802
3,2021-11-04,27778.803535,34800.0,59932.621088,142771.702910,103000.0,16769.094642,47076.849110,85275.621162,103000.0,...,49014.053101,52200.266267,82300.0,35422.766150,25725.495674,53517.645240,335000.0,261938.664056,27215.539268,17089.089434
4,2021-11-05,27764.960244,34800.0,59950.481531,142832.886577,103000.0,16751.262458,47113.523958,85703.786144,103000.0,...,49234.527906,52225.440281,82300.0,35296.978200,25803.917172,53327.130826,335000.0,261501.765215,27210.655072,17089.088512
5,2021-11-29,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
6,2021-11-30,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,2021-12-01,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,2021-12-02,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,2021-12-03,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [10]:

start_date = '20211101'
end_date = '20211105'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())

real=[]
real_data=pd.read_csv('C:\\Users\\user\\Desktop\\computing\\outside\\dacon\\stock_predict2\\open\\sample_submission.csv')
for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data['weekday'] = data.Date.apply(lambda x : x.weekday())
    data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
    data.Close = data.Close.ffill()
    data.Close = data.Close.bfill()

    real_data_close = pd.pivot_table(data = data, values = 'Close', columns = 'weekday', index = 'weeknum')
    real_data_close_flatten = np.ravel(real_data_close.to_numpy())
    # print(real_data_close_flatten)
    

    real_data.loc[:4,code]=real_data_close_flatten

type(real_data)
type(sample_submission)

label_np = real_data.to_numpy()[0:5,1:]
pred_np = sample_submission.to_numpy()[0:5, 1:]
print(pred_np)


result=np.mean(abs(label_np-pred_np)/label_np)*100
print(result)

WEEKDAY of "start_date" : 0
NUM of WEEKS to "end_date" : 44
HOW MANY "Business_days" : (5, 1)


,Date
0,2021-11-01
1,2021-11-02
2,2021-11-03
3,2021-11-04
4,2021-11-05


100%|██████████| 370/370 [01:05<00:00,  5.67it/s]

[[27537.914114823987 34800.0 59974.87857923791 ... 261561.32884400574
  27153.174712683995 17090.161904395427]
 [27742.68435843273 34800.0 60108.29645632018 ... 262163.4260609309
  27174.982867336483 17089.19402314418]
 [27802.818236518513 34800.0 60063.37042351088 ... 262425.8214172892
  27215.049381244324 17089.098802061344]
 [27778.803534601488 34800.0 59932.62108833865 ... 261938.66405647554
  27215.53926820737 17089.08943412018]
 [27764.960244338483 34800.0 59950.48153059793 ... 261501.76521533277
  27210.655072139743 17089.08851249317]]
35.69770501538279


In [11]:
# start_date = '20210101'
start_date = '20200607'
end_date = '20211126'

start_weekday = pd.to_datetime(start_date).weekday()
max_weeknum = pd.to_datetime(end_date).strftime('%V')
Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

print(f'WEEKDAY of "start_date" : {start_weekday}')
print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
print(f'HOW MANY "Business_days" : {Business_days.shape}', )
display(Business_days.head())
display(Business_days.tail())



WEEKDAY of "start_date" : 6
NUM of WEEKS to "end_date" : 47
HOW MANY "Business_days" : (385, 1)


,Date
0,2020-06-08
1,2020-06-09
2,2020-06-10
3,2020-06-11
4,2020-06-12


,Date
380,2021-11-22
381,2021-11-23
382,2021-11-24
383,2021-11-25
384,2021-11-26


In [12]:
# model = LinearRegression()
p_values=[0,1,2,3,4]
d_values=[1,2]
q_values=[0,1,2,3,4]
warnings.filterwarnings("ignore")

for code in tqdm(stock_list['종목코드'].values):
    data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
    data = pd.merge(Business_days, data, how = 'outer')
    data.Close = data.Close.ffill()
    data.Close = data.Close.bfill()
    data.index = data["Date"]
    arima_data = data['Close'][:]
    # arima_data = list(arima_data)
    

    predictions=evaluate_models(arima_data,p_values,d_values,q_values)    
    predictions=list(predictions)
    # print(predictions)
    # forecast = get_predict(arima_data) #(0,1,0)인수 나오는것 처리해야함
    # predictions = list(forecast)
    # print(predictions)

    sample_submission.loc[5:9,code] = predictions 
sample_submission.isna().sum().sum()

  0%|          | 0/370 [00:00<?, ?it/s]

ARIMA(0, 1, 0) AIC=6355.505
ARIMA(2, 1, 2) AIC=6353.652
Best ARIMA(2, 1, 2) AIC=6353.652


  0%|          | 1/370 [00:22<2:15:53, 22.09s/it]

[72214.42308887134, 72316.71792806406, 72509.30787009413, 72678.84125214306, 72752.92500751543]
ARIMA(0, 1, 0) AIC=6999.244


  1%|          | 2/370 [00:41<2:06:44, 20.66s/it]

...

 10%|█         | 38/370 [13:00<1:56:25, 21.04s/it]

Best ARIMA(0, 1, 0) AIC=7986.667
[520000.0, 520000.0, 520000.0, 520000.0, 520000.0]
ARIMA(0, 1, 0) AIC=7067.277
ARIMA(3, 1, 3) AIC=7066.950
ARIMA(4, 1, 4) AIC=7063.235
Best ARIMA(4, 1, 4) AIC=7063.235


 11%|█         | 39/370 [1:55:02<16:16:25, 177.00s/it]  

[93998.83147612364, 94284.34610841691, 93987.60379029656, 93629.42689577943, 93808.45113946246]


ConnectionError: HTTPSConnectionPool(host='fchart.stock.naver.com', port=443): Max retries exceeded with url: /sise.nhn?timeframe=day&count=6000&requestType=0&symbol=326030 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000192B32B8790>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [ ]:
sample_submission.to_csv('Final.csv',index=False)
sample_submission